### Handle imports:

In [19]:
# Move to root directory
import os

notebooks_dir = 'notebooks'
if notebooks_dir in os.path.abspath(os.curdir):
    while not os.path.abspath(os.curdir).endswith('notebooks'):
        print(os.path.abspath(os.curdir))
        os.chdir('..')
    os.chdir('..')  # to get to root

print(os.path.abspath(os.curdir))

/Users/jb_vandeneynde/code/jb-vde/Belfius/llm_esg_kpi/eyalytics


In [5]:
# Get API Key:
from dotenv import load_dotenv
load_dotenv()
if not os.getenv("OPENAI_API_KEY"):
    raise KeyError(
        "You will need an OPENAI_API_KEY to use the LLM models in this notebook."
    )

In [20]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import initialize_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.agents import Tool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

In [25]:
db = SQLDatabase.from_uri("sqlite:///data/cwyod/chinook.db")
toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0))

sql_agent = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [26]:
python_agent = create_python_agent(
    llm=OpenAI(temperature=0, max_tokens=1000),
    tool=PythonREPLTool(),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [39]:
agent = initialize_agent(
    tools=[
        Tool(
            name="SQLAgent",
            func=sql_agent.run,
            description="Useful for when you need to answer questions about music store. "
                        "You must not input SQL code in it, but the relevant part of the request. Use this more than the Python tool if the question "
                        "is about the music store.",
        ),
        Tool(
            name="PythonAgent",
            func=python_agent.run,
            description="""Useful to run python commands and for plotting.""",
        ),
    ],
    llm=OpenAI(temperature=0),
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [ ]:
# Standard prefix
prefix = "Fulfill the following request as best you can. You have access to the following tools:"

# Remind the agent of the Data tool, and what types of input it expects
suffix = (
    "Begin! When looking for data, do not write a SQL query. "
    "Pass the relevant portion of the request directly to the SQL tool in its entirety."
    "\n\n"
    "Request: {input}\n"
    "{agent_scratchpad}"
)


# Create the AgentExecutor
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [40]:
request = "Show me a bar chart of the evolution of total sales over the years."
response = agent.run(request)
print(response)



> Entering new AgentExecutor chain...
 I need to get the total sales for each year.
Action: SQLAgent
Action Input: SELECT YEAR(OrderDate) AS Year, SUM(TotalDue) AS TotalSales FROM Sales.SalesOrderHeader GROUP BY YEAR(OrderDate)

> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought: I should query the schema of the SalesOrderHeader table.
Action: sql_db_schema
Action Input: SalesOrderHeader
Observation: Error: table_names {'SalesOrderHeader'} not found in database
Thought: The table I need is Sales.SalesOrderHeader.
Action: sql_db_schema
Action Input: Sales.SalesOrderHeader
Observation: Error: table_names {'Sales.SalesOrderHeader'} not found in database
Thought: The table I need is Sales.SalesOrderHeader, not SalesOrderHeader.
Action: sql_db_schema
Action Input: Sales.SalesOrderHeader
Observation: Error: table_names {'Sales.SalesOrde

KeyboardInterrupt: 